# Generation Claim & Explanation

## Preliminary Operation

In [ ]:
import pandas as pd
import numpy as np
import openai
import ast
import os
import time

In [ ]:
#pd.set_option('display.max_rows', None)
#pd.set_option('display.width', 1000) 
pd.set_option('display.max_colwidth', None)

In [ ]:
openai.api_key = ''

In [ ]:
df_kb= pd.read_csv(r'df_triple_KB.csv')
df_kb.head()

In [ ]:
df_test = pd.read_csv(r'df_triple_lemma_TEST.csv')
df_test

### Prompt & Functions

* https://arxiv.org/abs/2201.11903 -> Link paper chain of thought

## New Claim

In [ ]:
def get_new_claim(system, user_template, claim, triple):
    formatted_triple =  ", ".join([f"{t['subject']} {t['predicate']} {t['object']}" for t in triple])
    user_filled = user_template.format(
        claim = claim,
        triple = formatted_triple
    )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",  # Specifica il modello 
        messages=[{"role": "system", "content": system},
                  {"role": "user", "content": user_filled}],
        temperature=0,
    )
    return response.choices[0].message['content'].strip()

In [ ]:
with open("system_claim.txt", "r") as f:
    system = f.read()
print(system)
with open("user_claim.txt", "r") as f:
    user = f.read()
print(user)

In [ ]:
for index, row in df_test.iterrows():
    if not row['triple_REBEL']:
        new_claim = "Non applicabile"
    else:
        triple_REBEL = ast.literal_eval(row['triple_REBEL'])
        try:
            new_claim = get_new_claim(system = system, user_template = user, claim = row['claim'], triple = triple_REBEL)
            print(new_claim)
            print("----------------------------------")   
            df_test.at[index, 'new_claim'] = new_claim
            time.sleep(1)
        except Exception as ex:
            print(ex)
            df_test.at[index, 'new_claim'] = "ERROR"

In [ ]:
for index, row in df_kb.iterrows():
    if not row['triple_REBEL']:
        new_claim = "Non applicabile"
    else:
        triple_REBEL = ast.literal_eval(row['triple_REBEL'])
        try:
            new_claim = get_new_claim(system = system, user_template = user, claim = row['claim'], triple = triple_REBEL)
            print(new_claim)
            print("----------------------------------")   
            df_kb.at[index, 'new_claim'] = new_claim
            time.sleep(1)
        except Exception as ex:
            df_kb.at[index, 'new_claim'] = "ERROR"

In [ ]:
df_kb.info()

## New Explanation

In [ ]:
def get_new_explanation(system, user_template, claim, explanation):
    user_filled = user_template.format(
        claim = claim,
        explanation = explanation
    )
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0125",  # Specifica il moodello
        messages=[{"role": "system", "content": system},
                  {"role": "user", "content": user_filled}],
        temperature=0,
    )
    return response.choices[0].message['content'].strip()

In [ ]:
with open("system_explanation.txt", "r") as f:
    system = f.read()
print(system)
with open("user_explanation.txt", "r") as f:
    user = f.read()
print(user)

In [ ]:
for index, row in df_test.iterrows():    
    try:
        new_expl = get_new_explanation(system = system, user_template = user, claim = row['claim'], explanation = row['explanation'])
        print(new_expl)
        print("----------------------------------")   
        df_test.at[index, 'new_expl'] = new_expl
    except Exception as ex:
        print(ex)
        df_test.at[index, 'new_expl'] = 'ERROR'

In [ ]:
df_test[['claim','new_claim','explanation', 'new_expl']].head(5)


In [ ]:
df_kb.head()

In [ ]:
for index, row in df_kb.iterrows():    
    try:
        new_expl = get_new_explanation(system = system, user_template = user, claim = row['claim'], explanation = row['explanation'])
        print(new_expl)
        print("----------------------------------")   
        df_kb.at[index, 'new_expl'] = new_expl
    except Exception as ex:
        print(ex)
        df_kb.at[index, 'new_expl'] = 'ERROR'

## Save Csv

In [ ]:
df_test.to_csv('df_test_generation.csv')

In [ ]:
df_kb.to_csv('df_kb_generation.csv')

In [ ]:
df_kb.info()